# Combine geodata with datasets

After extracting the data from multiple resources and converting it as csv, I can now combine the geometric data with the datasets.

I want to create two datasets, each of different area levels, namely province and municipality, and export it as csv.

First I import the datasets I need.

In [1]:
from pathlib import Path
from os import chdir
import pandas as pd

NL_POSTAL = pd.read_csv("../data/nl_postal.csv")
NL_PROVINCE_GEO = pd.read_csv("../data/cbs_provincie_2020_gegeneraliseerd.csv")
NL_MUNICIPALITY_GEO = pd.read_csv("../data/cbs_gemeente_2020_gegeneraliseerd.csv")
RDW_DATASET = pd.read_csv("../data/RDW_dataset.csv")

In [2]:
RDW_DATASET

,AreaManagerId,UsageIdDesc,StartDateUsageId,EndDateUsageId,SuperiorUsageId,AreaManagerDesc,AreaZone_Id,ExitPossibleAllDay,OpenAllYear,AreaDesc,UsageType_Id
0,484,Betaald Parkeren,20130919,29991231,PARKEREN,Alphen aan den Rijn,484_BOSPAR,False,False,Straatparkeren Paradijslaan (Alphen a/d Rijn),BETAALDP
1,484,Betaald Parkeren,20130919,29991231,PARKEREN,Alphen aan den Rijn,484_BOSPAR,False,False,Carpool Gemeneweg (Hazerswoude-Dorp),CARPOOL
2,484,Betaald Parkeren,20130919,29991231,PARKEREN,Alphen aan den Rijn,484_BOSPAR,False,False,Carpool Nieuwkoopseweg (Aarlanderveen),CARPOOL
3,484,Betaald Parkeren,20130919,29991231,PARKEREN,Alphen aan den Rijn,484_BOSPAR,False,False,Blauwe Zone Kerk en Zanen (Alphen a/d Rijn),BLAUWEZ
4,484,Betaald Parkeren,20130919,29991231,PARKEREN,Alphen aan den Rijn,484_BOSPAR,False,False,Vergunningen Cornelis de Vlamingstraat,VERGUNP
...,...,...,...,...,...,...,...,...,...,...,...
1042983,612,Garage Parkeren,20131010,20201019,PARKEREN,Spijkenisse,612_CPG,True,False,Terrein Van Houtenstraat en Vredehofstraat (ce...,TERREINP
1042984,612,Garage Parkeren,20131010,20201019,PARKEREN,Spijkenisse,612_CPG,True,False,Garage Stadhuis (Spijkenisse),GARAGEP
1042985,612,Garage Parkeren,20131010,20201019,PARKEREN,Spijkenisse,612_CPG,True,False,Garage Kolkplein (Spijkenisse),GARAGEP
1042986,612,Garage Parkeren,20131010,20201019,PARKEREN,Spijkenisse,612_CPG,True,False,Garage Boekenberg (Spijkenisse),GARAGEP


Then I have to filter the RDW-dataset. I need to filter the columns that I won't use, namely most of the *Id* columns.

In [3]:
def drop_df_columns(df, columns):
    return df.drop(df.columns[columns], axis=1)


drop_rdw_columns = [0, 2, 3, 4, 6, 9]
rdw = drop_df_columns(RDW_DATASET, drop_rdw_columns)

In [4]:
rdw

,UsageIdDesc,AreaManagerDesc,ExitPossibleAllDay,OpenAllYear,UsageType_Id
0,Betaald Parkeren,Alphen aan den Rijn,False,False,BETAALDP
1,Betaald Parkeren,Alphen aan den Rijn,False,False,CARPOOL
2,Betaald Parkeren,Alphen aan den Rijn,False,False,CARPOOL
3,Betaald Parkeren,Alphen aan den Rijn,False,False,BLAUWEZ
4,Betaald Parkeren,Alphen aan den Rijn,False,False,VERGUNP
...,...,...,...,...,...
1042983,Garage Parkeren,Spijkenisse,True,False,TERREINP
1042984,Garage Parkeren,Spijkenisse,True,False,GARAGEP
1042985,Garage Parkeren,Spijkenisse,True,False,GARAGEP
1042986,Garage Parkeren,Spijkenisse,True,False,GARAGEP


## Add related province and municipality

I can now merge the place name hierarchy dataset (NL_POSTAL) with the RDW-dataset. First I drop the columns I won't need. Then I rename the hierarchy dataset column to match the placename of the RDW dataset and merge them together.

In [5]:
rename_nl_col = {"placeName": "AreaManagerDesc"}
nl_place_hierarchy = drop_df_columns(NL_POSTAL, [3, 4])
nl_place_hierarchy = nl_place_hierarchy.rename(columns=rename_nl_col)
rdw_geo = pd.merge(rdw, nl_place_hierarchy, how="left", on=["AreaManagerDesc"])


In [6]:
rdw_geo

,UsageIdDesc,AreaManagerDesc,ExitPossibleAllDay,OpenAllYear,UsageType_Id,province,municipality
0,Betaald Parkeren,Alphen aan den Rijn,False,False,BETAALDP,Zuid-Holland,Alphen aan den Rijn
1,Betaald Parkeren,Alphen aan den Rijn,False,False,CARPOOL,Zuid-Holland,Alphen aan den Rijn
2,Betaald Parkeren,Alphen aan den Rijn,False,False,CARPOOL,Zuid-Holland,Alphen aan den Rijn
3,Betaald Parkeren,Alphen aan den Rijn,False,False,BLAUWEZ,Zuid-Holland,Alphen aan den Rijn
4,Betaald Parkeren,Alphen aan den Rijn,False,False,VERGUNP,Zuid-Holland,Alphen aan den Rijn
...,...,...,...,...,...,...,...
1044087,Garage Parkeren,Spijkenisse,True,False,TERREINP,Zuid-Holland,Nissewaard
1044088,Garage Parkeren,Spijkenisse,True,False,GARAGEP,Zuid-Holland,Nissewaard
1044089,Garage Parkeren,Spijkenisse,True,False,GARAGEP,Zuid-Holland,Nissewaard
1044090,Garage Parkeren,Spijkenisse,True,False,GARAGEP,Zuid-Holland,Nissewaard


The last things that I still need to do, before the dataset can be devided into municipality and province, is to calculate the sum of each usage type, and of the boolean values.

For the usage type I need to create a new column. With `groupby()` and `.transform()` I can count the sum of each usage type. In the transform function I use a anonymous function to count how many times each usage type appears in the dataset and save it in a new column called *UsageType_count*.

In [7]:
rdw_geo["UsageType_count"] = rdw_geo.groupby(["AreaManagerDesc", "UsageType_Id"])["UsageType_Id"].transform(lambda x: x.count())

In [8]:
rdw_geo

,UsageIdDesc,AreaManagerDesc,ExitPossibleAllDay,OpenAllYear,UsageType_Id,province,municipality,UsageType_count
0,Betaald Parkeren,Alphen aan den Rijn,False,False,BETAALDP,Zuid-Holland,Alphen aan den Rijn,1122
1,Betaald Parkeren,Alphen aan den Rijn,False,False,CARPOOL,Zuid-Holland,Alphen aan den Rijn,204
2,Betaald Parkeren,Alphen aan den Rijn,False,False,CARPOOL,Zuid-Holland,Alphen aan den Rijn,204
3,Betaald Parkeren,Alphen aan den Rijn,False,False,BLAUWEZ,Zuid-Holland,Alphen aan den Rijn,102
4,Betaald Parkeren,Alphen aan den Rijn,False,False,VERGUNP,Zuid-Holland,Alphen aan den Rijn,714
...,...,...,...,...,...,...,...,...
1044087,Garage Parkeren,Spijkenisse,True,False,TERREINP,Zuid-Holland,Nissewaard,60
1044088,Garage Parkeren,Spijkenisse,True,False,GARAGEP,Zuid-Holland,Nissewaard,150
1044089,Garage Parkeren,Spijkenisse,True,False,GARAGEP,Zuid-Holland,Nissewaard,150
1044090,Garage Parkeren,Spijkenisse,True,False,GARAGEP,Zuid-Holland,Nissewaard,150


## Calculate sum & divide data by municipality and province

I can now create the data required for the municipalities and provinces and seperate them.

To create the data for the municipality I have to count the sum of the boolean values that are `True` from the columns *ExitPossibleAllDay* and *OpenAllYear* and group by usage type. The sum will replace the boolean values.

To group and sum I created a function. I can use this function for grouping and sum the values for the province later.

In [9]:
def group_and_sum(df, group_columns, to_sum_columns):
    return df.groupby(group_columns, as_index=False)[to_sum_columns].sum()


mp_sum_cols = ["ExitPossibleAllDay", "OpenAllYear"]
mp_group_cols = [rdw_geo.columns.values[i] for i in [4, 5, 6, 7]]
mp_data = group_and_sum(rdw_geo, mp_group_cols, mp_sum_cols)

In [10]:
mp_data.loc[(mp_data["UsageType_Id"] == "BETAALDP") & (mp_data["province"] == "Noord-Holland")]
# mp_data

,UsageType_Id,province,municipality,UsageType_count,ExitPossibleAllDay,OpenAllYear
65,BETAALDP,Noord-Holland,Alkmaar,1254,1254,1254
66,BETAALDP,Noord-Holland,Amstelveen,203,203,203
67,BETAALDP,Noord-Holland,Amsterdam,43785,41700,35445
68,BETAALDP,Noord-Holland,Bloemendaal,16,16,16
69,BETAALDP,Noord-Holland,Den Helder,96,96,96
70,BETAALDP,Noord-Holland,Diemen,60,60,60
71,BETAALDP,Noord-Holland,Enkhuizen,108,108,108
72,BETAALDP,Noord-Holland,Gooise Meren,28,28,28
73,BETAALDP,Noord-Holland,Gooise Meren,30,30,30
74,BETAALDP,Noord-Holland,Haarlem,120,120,120


As you can see in the result above, when I filter on *Noord-Holland* and usage type *BETAALDP* the municipality **Gooise Meren** appears twice. Apparently not all municipalities got grouped together correctly. To merge the municipalities correctly I grouped them again by municipality.

In [11]:
mp_all_summed = mp_data.groupby(["municipality", "UsageType_Id", "province"], as_index=False).sum()

In [12]:
mp_all_summed.loc[(mp_data["UsageType_Id"] == "BETAALDP") & (mp_data["province"] == "Noord-Holland")]
# mp_data

,municipality,UsageType_Id,province,UsageType_count,ExitPossibleAllDay,OpenAllYear
65,Apeldoorn,STADSBREED,Gelderland,56,42,49
66,Apeldoorn,VERGUNP,Gelderland,560,420,490
67,Appingedam,PARKRIDE,Groningen,4,4,4
68,Arnhem,BETAALDP,Gelderland,3744,3328,2080
69,Arnhem,BEZOEKP,Gelderland,648,576,360
70,Arnhem,CARPOOL,Gelderland,72,64,40
71,Arnhem,GARAGEP,Gelderland,360,320,200
72,Arnhem,PARKEREN,Gelderland,216,192,120
73,Arnhem,PARKRIDE,Gelderland,288,256,160
74,Arnhem,VERGUNP,Gelderland,1008,896,560


Now **Gooise Meren** is grouped correctly showing the right sum.

## Add missing municipalities

Next I want to add the missing municipalities that don't have any data available.

First I use the postal dataset and remove the column with place names. Then I merge it with the municipality dataset on *municipality* and *province*. That gives me the data of both datasets in one dataset. Because multiple entries of the postal dataset are the same I remove any duplicates in the dataset, also the entries from the municipality dataset. The last thing I do is reset the index and it's complete.

Now I have a dataset with all the municipalities and the corresponding data.

In [13]:
nl_municipalities = drop_df_columns(nl_place_hierarchy, [0])
mp_data = pd.merge(mp_all_summed, nl_municipalities, how="outer", on=["municipality", "province"])
mp_data = mp_data.drop_duplicates()
mp_data = mp_data.reset_index(drop=True)

In [14]:
mp_data

,municipality,UsageType_Id,province,UsageType_count,ExitPossibleAllDay,OpenAllYear
0,'s-Hertogenbosch,BETAALDP,Noord-Brabant,11748.0,10680.0,7476.0
1,'s-Hertogenbosch,CARPOOL,Noord-Brabant,198.0,180.0,126.0
2,'s-Hertogenbosch,GARAGEP,Noord-Brabant,330.0,300.0,210.0
3,'s-Hertogenbosch,PARKRIDE,Noord-Brabant,264.0,240.0,168.0
4,Aalten,PARKRIDE,Gelderland,5.0,5.0,5.0
...,...,...,...,...,...,...
798,Weesp,NaN,Noord-Holland,NaN,NaN,NaN
799,Westervoort,NaN,Gelderland,NaN,NaN,NaN
800,Woudenberg,NaN,Utrecht,NaN,NaN,NaN
801,Zeewolde,NaN,Flevoland,NaN,NaN,NaN


With the data of the municipalities now complete, the only thing I have to do to create the province data is to sum by province. 

In [15]:
pv_sum_col = ["ExitPossibleAllDay", "OpenAllYear", "UsageType_count"]
pv_group_cols = [rdw_geo.columns.values[i] for i in [4, 5]]
pv_data = group_and_sum(mp_all_summed, pv_group_cols, pv_sum_col)

In [16]:
pv_data.loc[(pv_data["UsageType_Id"] == "BETAALDP")]

,UsageType_Id,province,ExitPossibleAllDay,OpenAllYear,UsageType_count
12,BETAALDP,Drenthe,5948,5252,6318
13,BETAALDP,Flevoland,2034,1944,2034
14,BETAALDP,Friesland,1760,1370,2362
15,BETAALDP,Gelderland,18319,16018,20928
16,BETAALDP,Groningen,3107,2872,3205
17,BETAALDP,Limburg,12421,8543,12477
18,BETAALDP,Noord-Brabant,32506,25556,41890
19,BETAALDP,Noord-Holland,44715,38400,47402
20,BETAALDP,Overijssel,32161,29671,32161
21,BETAALDP,Utrecht,22333,23290,23830


## Add geometry data to municipalities and provinces

Now the data is divided into municipalities and provinces I can add the associated geometry.

First I have to rename the column again to merge with and then I can merge the municipality geo data.

In [17]:
rename_to_mp = {"statnaam": "municipality"}
mp_geo = NL_MUNICIPALITY_GEO.rename(columns=rename_to_mp)
mp_complete = pd.merge(mp_data, mp_geo, how="left", on=["municipality"])

In [18]:
mp_complete

,municipality,UsageType_Id,province,UsageType_count,ExitPossibleAllDay,OpenAllYear,geometry
0,'s-Hertogenbosch,BETAALDP,Noord-Brabant,11748.0,10680.0,7476.0,MULTIPOLYGON (((5.444026282962954 51.753312234...
1,'s-Hertogenbosch,CARPOOL,Noord-Brabant,198.0,180.0,126.0,MULTIPOLYGON (((5.444026282962954 51.753312234...
2,'s-Hertogenbosch,GARAGEP,Noord-Brabant,330.0,300.0,210.0,MULTIPOLYGON (((5.444026282962954 51.753312234...
3,'s-Hertogenbosch,PARKRIDE,Noord-Brabant,264.0,240.0,168.0,MULTIPOLYGON (((5.444026282962954 51.753312234...
4,Aalten,PARKRIDE,Gelderland,5.0,5.0,5.0,MULTIPOLYGON (((6.559352360678049 51.974364063...
...,...,...,...,...,...,...,...
798,Weesp,NaN,Noord-Holland,NaN,NaN,NaN,MULTIPOLYGON (((5.044395028764473 52.327148857...
799,Westervoort,NaN,Gelderland,NaN,NaN,NaN,MULTIPOLYGON (((5.974910166168503 51.941620158...
800,Woudenberg,NaN,Utrecht,NaN,NaN,NaN,MULTIPOLYGON (((5.484301742753559 52.105542117...
801,Zeewolde,NaN,Flevoland,NaN,NaN,NaN,MULTIPOLYGON (((5.508991604739609 52.397662414...


I do the same with the province geo data.

In [19]:
rename_to_pv = {"statnaam": "province"}
pv_geo = NL_PROVINCE_GEO.rename(columns=rename_to_pv)
pv_complete = pd.merge(pv_data, pv_geo, how="left", on=["province"])


In [20]:
pv_complete

,UsageType_Id,province,ExitPossibleAllDay,OpenAllYear,UsageType_count,geometry
0,5DAAGSE,Noord-Brabant,1122,1122,2244,MULTIPOLYGON (((5.5356338752006 51.81657437309...
1,ALLES,Friesland,63,63,63,MULTIPOLYGON (((6.176870914534096 53.414654753...
2,AUTODATE,Limburg,154,98,154,MULTIPOLYGON (((5.932766879663456 51.741938447...
3,AUTODP,Zuid-Holland,336,432,432,MULTIPOLYGON (((4.54732226027155 52.3139965981...
4,AUTOLUW,Limburg,99,88,99,MULTIPOLYGON (((5.932766879663456 51.741938447...
...,...,...,...,...,...,...
189,WERKNMRP,Zuid-Holland,2688,3456,3456,MULTIPOLYGON (((4.54732226027155 52.3139965981...
190,ZAKE,Noord-Brabant,572,572,572,MULTIPOLYGON (((5.5356338752006 51.81657437309...
191,ZORG,Noord-Brabant,52,52,52,MULTIPOLYGON (((5.5356338752006 51.81657437309...
192,ZORGP,Zuid-Holland,168,216,216,MULTIPOLYGON (((4.54732226027155 52.3139965981...


## Export as csv

The last thing is to export the two datasets. I use a function I made earlier to export a DataFrame as csv

In [21]:
def write_csv(data, filename):
    Path("data").mkdir(parents=True, exist_ok=True)
    chdir("data")
    file = f"{filename}.csv"
    data.to_csv(file)
    print(file, "is created in folder: data")
    chdir("../")


write_csv(pv_complete, "province")
write_csv(mp_complete, "municipality")

province.csv is created in folder: data
municipality.csv is created in folder: data
